In [1]:
dbutils.library.installPyPI("boto3", version="1.9.157")
dbutils.library.restartPython()

PyPI package boto3 has been installed already. The previous installed package is PyPI:(boto3)-(1.9.157)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.

In [2]:
# import AWS credentials
# import config.py ##for local
%run "/dbfs/FileStore/tables/config" ##for databricks

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
from pyspark.sql.types import DateType, IntegerType
import pyspark.sql.functions as F

In [4]:
# get or create Spark session

app_name = "spark-postgres"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [5]:
# file to transform and load after uploading to DBFS
amazon_reviews_file = "/FileStore/tables/amazon_reviews_us_Software_v1_00_tsv-16dc0.gz"
amazon_reviews_file

Out[4]: '/FileStore/tables/amazon_reviews_us_Software_v1_00_tsv-16dc0.gz'

In [6]:
df_reviews = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .option("delimiter", "\t") \
    .load(amazon_reviews_file)
df_reviews.printSchema()

root
-- marketplace: string (nullable = true)
-- customer_id: integer (nullable = true)
-- review_id: string (nullable = true)
-- product_id: string (nullable = true)
-- product_parent: integer (nullable = true)
-- product_title: string (nullable = true)
-- product_category: string (nullable = true)
-- star_rating: integer (nullable = true)
-- helpful_votes: integer (nullable = true)
-- total_votes: integer (nullable = true)
-- vine: string (nullable = true)
-- verified_purchase: string (nullable = true)
-- review_headline: string (nullable = true)
-- review_body: string (nullable = true)
-- review_date: timestamp (nullable = true)

In [7]:
print("This dataset has " + str(df_reviews.count()) + " rows.")

This dataset has 341931 rows.

In [8]:
## Transform data to schema specifications

# review_id_table
review_id_df = df_reviews.select("review_id", "customer_id", "product_id", "product_parent", "review_date") \
                        .withColumn('review_date', F.date_format('review_date', "yyyy-MM-dd").cast(DateType()))

# products table
# need to select unique values due to PK constraint so aggregate and drop count column
products_df = df_reviews.groupBy("product_id", "product_title").agg(F.count("product_id")).select("product_id", "product_title")

# customers table
customers_df = df_reviews.groupBy("customer_id").agg(F.count("review_id").cast(IntegerType()).alias("customer_count"))

# vine table
vine_df = df_reviews.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")

In [9]:
import boto3

secret_name = my_secret_name
region_name = my_region_name
access_key = my_access_key
secret_key = my_secret_key

session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)
secret_value

Out[8]: {'ARN': 'arn:aws:secretsmanager:us-east-2:436648248389:secret:ut/hw22/postgres-VP2diM',
 'Name': 'ut/hw22/postgres',
 'VersionId': 'c7e5dc60-b7c0-446b-a18c-05b451d3c640',
 'SecretString': '{"username":"postgres","password":"changeme","engine":"postgres","host":"hw22.c5mqc4efqglg.us-east-2.rds.amazonaws.com","port":5432,"dbInstanceIdentifier":"hw22"}',
 'VersionStages': ['AWSCURRENT'],
 'CreatedDate': datetime.datetime(2019, 12, 28, 4, 36, 26, 847000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c0e50585-bf2d-4146-925b-0579742b732b',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'date': 'Tue, 31 Dec 2019 04:31:09 GMT',
 'content-type': 'application/x-amz-json-1.1',
 'content-length': '427',
 'connection': 'keep-alive',
 'x-amzn-requestid': 'c0e50585-bf2d-4146-925b-0579742b732b'},
 'RetryAttempts': 0}}

In [10]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])
get_connection(secret_value)

Out[9]: {'username': 'postgres',
 'password': 'changeme',
 'engine': 'postgres',
 'host': 'hw22.c5mqc4efqglg.us-east-2.rds.amazonaws.com',
 'port': 5432,
 'dbInstanceIdentifier': 'hw22'}

In [11]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}
jdbcUrl

Out[10]: 'jdbc:postgresql://hw22.c5mqc4efqglg.us-east-2.rds.amazonaws.com:5432/postgres'

In [12]:
mode = "append" # options are: error, append, overwrite

review_id_df.write.jdbc(jdbcUrl, "sw_review_id_table", mode, connectionProperties)
products_df.write.jdbc(jdbcUrl, "sw_products", mode, connectionProperties)
customers_df.write.jdbc(jdbcUrl, "sw_customers", mode, connectionProperties)
vine_df.write.jdbc(jdbcUrl, "sw_vine_table", mode, connectionProperties)